In [26]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
file_path = 'sample_data for DE assessment.csv'  # Update this path as needed
df = pd.read_csv(file_path,na_values=[''], keep_default_na=False)



In [27]:
df.head(5)

,timestamp,user_id,event_type,region,country,device_id,referrer
0,1/9/2022 0:00,2790,ordering_page,NA,UNITED STATES,1475,NaN
1,1/9/2022 0:00,2513,ordering_page,NA,UNITED STATES,50,google
2,1/9/2022 0:00,2513,ordering_page,NA,UNITED STATES,50,google
3,1/9/2022 0:00,2790,home_page,NA,UNITED STATES,1475,NaN
4,1/9/2022 0:00,1059,ordering_page,JP,JAPAN,2803,NaN


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33123 entries, 0 to 33122
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   timestamp   33123 non-null  object
 1   user_id     33123 non-null  int64 
 2   event_type  33123 non-null  object
 3   region      33123 non-null  object
 4   country     33008 non-null  object
 5   device_id   33123 non-null  int64 
 6   referrer    15357 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.8+ MB


In [30]:
#df = pd.DataFrame(data)

# Convert 'timestamp' column to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Convert 'user_id' and 'device_id' columns to int64 (if not already)
df['user_id'] = df['user_id'].astype('int64')
df['device_id'] = df['device_id'].astype('int64')

# Convert 'event_type', 'region', 'country', and 'referrer' columns to string (object)
df['event_type'] = df['event_type'].astype('str')
df['region'] = df['region'].astype('str')
df['country'] = df['country'].astype('str')
df['referrer'] = df['referrer'].astype('str')

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33123 entries, 0 to 33122
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   timestamp   33123 non-null  datetime64[ns]
 1   user_id     33123 non-null  int64         
 2   event_type  33123 non-null  object        
 3   region      33123 non-null  object        
 4   country     33123 non-null  object        
 5   device_id   33123 non-null  int64         
 6   referrer    33123 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 1.8+ MB


In [32]:
# Before de-duplication
df[df['user_id']==13]

,timestamp,user_id,event_type,region,country,device_id,referrer
22271,2022-01-13 18:39:00,13,ordering_page,EMEA,UNITED KINGDOM,1850,nan
22293,2022-01-13 18:43:00,13,ordering_page,EMEA,UNITED KINGDOM,1850,nan
22299,2022-01-13 18:43:00,13,checkout_page,EMEA,UNITED KINGDOM,1850,nan
22301,2022-01-13 18:44:00,13,checkout_success,EMEA,UNITED KINGDOM,1850,nan


In [33]:
# Drop duplicates to ensure each user-event pair is counted once
df = df.drop_duplicates(subset=['user_id', 'event_type'])


In [34]:
# Checking out after de-duplication
df[df['user_id']==13]

,timestamp,user_id,event_type,region,country,device_id,referrer
22271,2022-01-13 18:39:00,13,ordering_page,EMEA,UNITED KINGDOM,1850,nan
22299,2022-01-13 18:43:00,13,checkout_page,EMEA,UNITED KINGDOM,1850,nan
22301,2022-01-13 18:44:00,13,checkout_success,EMEA,UNITED KINGDOM,1850,nan


In [35]:
# Count unique users at each stage of the funnel
home_page_visits = df[df['event_type'] == 'home_page']['user_id'].nunique()
order_page_visits = df[df['event_type'] == 'ordering_page']['user_id'].nunique()
checkout_page_visits = df[df['event_type'] == 'checkout_page']['user_id'].nunique()
successful_checkouts = df[df['event_type'] == 'checkout_success']['user_id'].nunique()


In [36]:
print(home_page_visits,order_page_visits,checkout_page_visits,successful_checkouts)

2593 2090 1058 939


In [11]:
# Calculate conversion rates
home_to_order_rate = (order_page_visits / home_page_visits) * 100 if home_page_visits else 0
order_to_checkout_rate = (checkout_page_visits / order_page_visits) * 100 if order_page_visits else 0
checkout_to_success_rate = (successful_checkouts / checkout_page_visits) * 100 if checkout_page_visits else 0


In [12]:

# Display conversion rates in percentage format
print(f"Conversion rate from home page visits to order page visits: {home_to_order_rate:.2f}%")
print(f"Conversion rate from order page visits to checkout page visits: {order_to_checkout_rate:.2f}%")
print(f"Conversion rate from checkout page visits to successful checkout page visits: {checkout_to_success_rate:.2f}%")

Conversion rate from home page visits to order page visits: 80.60%
Conversion rate from order page visits to checkout page visits: 50.62%
Conversion rate from checkout page visits to successful checkout page visits: 88.75%


out of  80.60% only 50.62% are going to checkout page and out this 50.62% only 88.75% are going to successful page